In [1]:
import os
import pickle


In [2]:
def all_path(dirname, filter_list):
    result = []  # 所有的文件
    # filter = [".wav"]
    for maindir, subdir, file_name_list in os.walk(dirname):
        for filename in file_name_list:
            apath = os.path.join(maindir, filename)  # 合并成一个完整路径
            ext = os.path.splitext(apath)[1]
            if ext in filter_list:
                result.append(apath)
    return result

In [3]:
sols = all_path("/home/jrj/postgraduate/backdoor_data/dataset/real_world_v2", [".sol"])
print(len(sols))

In [6]:
jsons = all_path("/home/jrj/postgraduate/backdoor_data/result/new_real_world_v2", [".json"])
print(len(jsons))

In [ ]:
v1_folders = os.listdir("/home/jrj/postgraduate/Symbolic/Backdoor/result/real_world/0.4.x-0.5.x")+os.listdir("/home/jrj/postgraduate/Symbolic/Backdoor/result/real_world/0.6.x")
v2_folders = os.listdir("/home/jrj/postgraduate/backdoor_data/dataset/real_world_selected_v2")
r = v1_folders in v2_folders

In [ ]:
r = set(v1_folders).difference(v2_folders)

In [ ]:
with open('/home/jrj/postgraduate/Symbolic/Backdoor/scripts/error_list.pkl', 'rb') as fr:
    a = pickle.load(fr)

In [ ]:
from solidity_parser import parse_file

In [ ]:
soure_unit = parse_file('/home/jrj/postgraduate/Symbolic/Backdoor/dataset/verified/GenerateToken/0xa8ba86e4a8dd2afecd6ee4e6b56d6fc8c8864f09.sol', loc=True)['children']

In [ ]:
import requests
import eventlet
import time

eventlet.monkey_patch()

time_limit = 3  #set timeout time 3s

with eventlet.Timeout(time_limit,False):
    time.sleep(5)
    print('error')
print("over")

In [ ]:
import json
import pickle
import os
from tqdm import tqdm

In [ ]:
type_list = ["arbitrarilyTransfer", "DestroyToken", "DisableTransfer", "FreezeAccount", "GenerateToken"]

In [ ]:

for backdoor_type in type_list:
    files = all_path(f'/home/jrj/postgraduate/Symbolic/Backdoor/dataset/verified/modified/{backdoor_type}', [".sol"])
    error_list = []
    for file_path in tqdm(files):
        old_filename = os.path.basename(file_path)[:-4]
        new_filename = old_filename + '.json'
        path = f"/home/jrj/postgraduate/Symbolic/Backdoor/result/verified/{backdoor_type}"
        new_path = os.path.join(path, new_filename)
        if os.path.exists(new_path):
            if os.path.getsize(new_path) == 2:
                error_list.append(file_path)
        else:
            error_list.append(file_path)
    with open(f'error_list_{backdoor_type}.txt', 'w') as fw:
        fw.write("\n".join(error_list))

In [ ]:

for backdoor_type in type_list:
    sh_content = "#！/bin/zsh\n"
    files = all_path(f'/home/jrj/postgraduate/Symbolic/Backdoor/dataset/verified/modified/{backdoor_type}', [".sol"])
    for file in files:
        python_content = f'python validate_function.py "{file}" "{backdoor_type}"\n'
        sh_content += python_content
    with open(f'../validation/{backdoor_type}.sh', 'w') as fw:
        fw.write(sh_content)

In [ ]:
root_path = "/home/jrj/postgraduate/Symbolic/Backdoor/dataset/real_world_selected/0.4.x-0.5.x"
folder_list = os.listdir(root_path)
num = 0
round = 0
batch_size = 200
while num < len(folder_list):
    if num % batch_size == 0:
        round = int(num / batch_size)
        sh_content = "#！/bin/zsh\n"
    folder = folder_list[num]
    src_path = os.path.join(root_path, folder)
    meta_file = os.path.join(src_path, "meta.json")
    with open(meta_file, 'r') as fr:
            meta_dict = json.load(fr)
    sol_path = os.path.join(src_path, f"{meta_dict['Contract Name']}_inline.sol")
    solc_path = f"/home/jrj/.solc-select/artifacts/solc-{meta_dict['Version']}/solc-{meta_dict['Version']}"
    python_content = f'/home/jrj/.conda/envs/slither/bin/python ../validate_function.py "{sol_path}" "{solc_path}"\n'
    sh_content += python_content
    if (num + 1) % batch_size == 0 or (num+1)>=len(folder_list):
        with open(f'../validation/0.4.x-0.5.x/real_world_{str(round)}.sh', 'w') as fw:
            fw.write(sh_content)
    num += 1

In [ ]:
for i in range(15):
    content = f"nohup time ./real_world_{str(i)}.sh >real_world_{str(i)}.log 2>&1 &"
    print(content)

In [ ]:
root_path = "/home/jrj/postgraduate/Symbolic/Backdoor/dataset/real_world_selected/0.6.x"
folder_list = os.listdir(root_path)
num = 0
round = 0
batch_size = 200
while num < len(folder_list):
    if num % batch_size == 0:
        round = int(num / batch_size)
        sh_content = "#！/bin/zsh\n"
    folder = folder_list[num]
    src_path = os.path.join(root_path, folder)
    meta_file = os.path.join(src_path, "meta.json")
    with open(meta_file, 'r') as fr:
            meta_dict = json.load(fr)
    sol_path = os.path.join(src_path, f"{meta_dict['Contract Name']}_inline.sol")
    solc_path = f"/home/jrj/.solc-select/artifacts/solc-{meta_dict['Version']}/solc-{meta_dict['Version']}"
    python_content = f'/home/jrj/.conda/envs/slither/bin/python ../validate_function.py "{sol_path}" "{solc_path}"\n'
    sh_content += python_content
    if (num + 1) % batch_size == 0 or (num+1)>=len(folder_list):
        with open(f'../validation/0.6.x/real_world_0.6_{str(round)}.sh', 'w') as fw:
            fw.write(sh_content)
    num += 1

In [ ]:
for i in range(15):
    content = f"nohup time ./real_world_0.6_{str(i)}.sh >real_world_0.6_{str(i)}.log 2>&1 &"
    print(content)

In [ ]:
import os
import shutil

In [ ]:
folders = os.listdir("/home/jrj/postgraduate/Symbolic/Backdoor/dataset/real_world_selected")

for folder in folders:
    if not folder.startswith("0x"):continue
    origin_folder_path = os.path.join("/home/jrj/postgraduate/Symbolic/Backdoor/dataset/real_world_selected", folder)
    new_folder_path = os.path.join("/home/jrj/postgraduate/Symbolic/Backdoor/dataset/real_world_selected/0.4.x-0.5.x", folder)
    shutil.move(origin_folder_path, new_folder_path)

In [ ]:
path = '/home/jrj/postgraduate/Symbolic/etherscan-contract-crawler/contracts/contracts_20220808_130510.json'
with open(path, 'r') as fr:
    json_dict = json.load(fr)

In [ ]:
json_dict